In [13]:
import numpy as np
import pandas as pd
from main import MLP_Classifier,Layer
from sklearn.datasets import make_classification
import optuna
from sklearn.metrics import accuracy_score,precision_score,recall_score
from sklearn.model_selection import train_test_split

/opt/python/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
np.random.seed(42)
# Generate  dataset
X, Y = make_classification(
    n_samples=1000,     
    n_features=4,       
    n_redundant=0,      
    n_clusters_per_class=1,
    flip_y=0.1,         # Add label noise
    class_sep=1.0,      # Class separation
    n_classes=2,      # nb classes
)


noise = np.random.normal(0, 0.5, X.shape)
X = X + noise
X=pd.DataFrame(X)
Y=pd.Series(Y)
if len(np.unique(Y))==2:
   Y=pd.DataFrame(Y)
else: 
   Y=pd.get_dummies(Y).astype(int)


determine objective function to optimise :  minimise cross entropy (in pdf maximise log-likelihood).

for example we optimise over batch size, learning rate and dropout (one of the most important parameters in NN).

we could do also on layers but computantionally expensive for large datasets.


In [15]:



def objective(trial):
    # Define hyperparameter search space
    batch_size = trial.suggest_int("batch_size", 500, 800)
    alpha = trial.suggest_float("alpha", 0.01, 0.1)
    dropout_rate = trial.suggest_float("dropout", 0.5, 0.9)

    model = MLP_Classifier(
        (
            (
                Layer(
                    nb_neurons=20,
                    activation_function="relu",
                    regul=("l2", 0.1),
                    initial="he",
                    batchnorm=True
                ),
                Layer(
                    nb_neurons=10,
                    activation_function="relu",
                    regul=("l2", 0.1),
                    initial="he",
                ),
                Layer(
                    nb_neurons=30,
                    activation_function="relu",
                    regul=("dropout", dropout_rate),
                    initial="he",
                ),
            )
        ),
        max_iter=2000,
        thr=1e-5,
        alpha=alpha,
        seed=123,
        batch_size=batch_size,
        verbose=False,
        optim="adam"
    )

    model.train(X, Y)

    score = model.loss(Y,model.y_hat)  # need to do on val set

    return score


storage = "sqlite:///optuna_mlpsoftmax.db"
study = optuna.create_study(
    direction="minimize", study_name="MLP", storage=storage, load_if_exists=True
)  # 'minimize' for loss functions
study.optimize(objective, n_trials=1)

print("Best Hyperparameters:", study.best_params)


[I 2026-01-04 19:50:55,976] A new study created in RDB with name: MLP
[I 2026-01-04 19:50:57,713] Trial 0 finished with value: 0.3785585541635659 and parameters: {'batch_size': 752, 'alpha': 0.02319128823729845, 'dropout': 0.5788575266859909}. Best is trial 0 with value: 0.3785585541635659.


Best Hyperparameters: {'batch_size': 752, 'alpha': 0.02319128823729845, 'dropout': 0.5788575266859909}


In [16]:
best_results={"best value" : study.best_trial.values,"params": study.best_trial.params}
best_results


{'best value': [0.3785585541635659],
 'params': {'batch_size': 752,
  'alpha': 0.02319128823729845,
  'dropout': 0.5788575266859909}}

run model on optimised parameters

In [17]:


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

model = MLP_Classifier(
    (
        (
            Layer(
                nb_neurons=20,
                activation_function="relu",
                regul=("l2", 0.1),
                initial="he",
                batchnorm=True
         
            ),
            Layer(
                nb_neurons=10,
                activation_function="relu",
                regul=("l2", 0.1),
                initial="he",
                
            ),
            Layer(
                nb_neurons=30,
                activation_function="relu",
                regul=("dropout", best_results["params"]["dropout"]),
                initial="he",
                
            ),
        )
    ),
    max_iter=2000,
    thr=1e-5,
    alpha=best_results["params"]["alpha"],
    seed=123,
    batch_size=best_results["params"]["batch_size"],
    verbose=True,
    optim="adam",
    nb_epochs_early_stopping=50
)

fct=accuracy_score

model.train(X_train, y_train,X_test,y_test,fct)

print(f"final {fct.__name__}", accuracy_score(model.predict(X_test), y_test))


-------------------------------------------------------------------------
iteration 0 : TRAIN accuracy_score  : 0.7093333333333334, loss : 0.6566102261215156
iteration 0 : TEST accuracy_score  : 0.748, loss : 0.6565915727919444
-------------------------------------------------------------------------
iteration 50 : TRAIN accuracy_score  : 0.8133333333333334, loss : 0.43423980488257347
iteration 50 : TEST accuracy_score  : 0.828, loss : 0.41263360186056397
-------------------------------------------------------------------------
iteration 100 : TRAIN accuracy_score  : 0.8186666666666667, loss : 0.4144917150850996
iteration 100 : TEST accuracy_score  : 0.852, loss : 0.39384797691126466
-------------------------------------------------------------------------
iteration 150 : TRAIN accuracy_score  : 0.8306666666666667, loss : 0.40984694893046175
iteration 150 : TEST accuracy_score  : 0.86, loss : 0.3658694271920856
early stopping at epoch 161
final accuracy_score 0.888


In [19]:
model.E.keys()

dict_keys([3, 2, 1])

In [7]:
model.nb_layers

3

In [8]:
model.network

{1: {'layer_type': layers.Layer,
  'nb_neurons': 20,
  'activ_fct': 'relu',
  'regul': 'l2',
  'regul_param': 0.1,
  'init': 'he',
  'law': 'normal',
  'batchnorm': True},
 2: {'layer_type': layers.Layer,
  'nb_neurons': 10,
  'activ_fct': 'relu',
  'regul': 'l2',
  'regul_param': 0.1,
  'init': 'he',
  'law': 'normal',
  'batchnorm': False},
 3: {'layer_type': layers.Layer,
  'nb_neurons': 30,
  'activ_fct': 'relu',
  'regul': 'dropout',
  'regul_param': 0.8375822042749481,
  'init': 'he',
  'law': 'normal',
  'batchnorm': False}}

In [1]:
from layers import ConvLayer,MaxPoolLayer,Layer,FlatLayer
from cnn import CNN
import numpy as np 

In [13]:

q=CNN(

    (
        ConvLayer(in_channels=3,output_channels=16,kernel_size=2,stride=1,padding=True,activation_function="relu",initial="lecun",law="normal"),
        MaxPoolLayer(kernel_size=3,stride=2,padding=True),
        ConvLayer(in_channels=16,output_channels=32,kernel_size=2,stride=1,padding=True,activation_function="relu",initial="lecun",law="normal"),
        MaxPoolLayer(kernel_size=3,stride=2,padding=True),
        ConvLayer(in_channels=32,output_channels=64,kernel_size=2,stride=1,padding=True,activation_function="relu",initial="lecun",law="normal"),
        MaxPoolLayer(kernel_size=3,stride=2,padding=True),
        
        FlatLayer(),
        Layer(
                nb_neurons=20,
                activation_function="relu",
                regul=("l2", 0.1),
                initial="he",
                batchnorm=True
         
            ),
            Layer(
                nb_neurons=10,
                activation_function="relu",
                regul=("l2", 0.1),
                initial="he",
                
            ),
            Layer(
                nb_neurons=30,
                activation_function="relu",
                regul=("dropout",0.5),
                initial="he",
                
            ),
        
    ),
    max_iter=2000,
    thr=1e-5,
    alpha=0.001,
    seed=123,
    batch_size=200,
    verbose=True,
    nb_epochs_early_stopping=20



)

In [14]:
X=np.random.randn(20, 20, 3,10)
Y=np.random.choice([0, 1], size=(10,1))


In [15]:
q.train(X,Y)

dummy res shape (1, 64)


In [16]:
q.nb_cnn_layers

7

In [17]:
q.nb_layers

3

In [18]:
q.predict(X)

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

In [19]:
q.forward_cnn(X,"train")

array([[0.12667004, 3.89158695, 0.73909931, 0.92356768, 1.46190637,
        2.86896681, 0.43592179, 0.44208886, 0.54217387, 0.        ,
        1.09392343, 1.16337488, 0.        , 1.82052638, 0.        ,
        2.15616352, 0.        , 0.        , 0.        , 0.        ,
        0.40687296, 0.70889481, 0.        , 0.        , 0.        ,
        0.93463105, 0.01786255, 0.73793597, 0.4328405 , 0.        ,
        0.        , 1.11582354, 0.        , 0.        , 0.        ,
        0.        , 0.76664667, 0.        , 0.        , 0.9349631 ,
        0.28449726, 0.73353958, 0.83110961, 2.88449204, 0.        ,
        0.20571789, 0.1875448 , 0.        , 0.89604675, 1.86586614,
        0.        , 0.40452998, 0.        , 0.        , 0.62401816,
        1.14414384, 0.        , 1.20756189, 0.05719768, 0.12704129,
        0.6667232 , 0.73491228, 0.        , 1.64055764],
       [0.        , 3.81690131, 0.70795512, 0.71252831, 1.34830024,
        2.95381106, 0.        , 0.221113  , 0.59620179, 0. 

In [20]:
q.test(X,Y)

7 FlatLayer
6 MaxPoolLayer
5 ConvLayer
4 MaxPoolLayer
3 ConvLayer
2 MaxPoolLayer
1 ConvLayer
